# “Dogs vs. Cats” Challenge

# Downloading dataset from Kaggle

In [ ]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

In [ ]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [ ]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats

In [ ]:
!unzip dogs-vs-cats.zip

In [ ]:
from zipfile import ZipFile

file_name = "train.zip"

# opening the zip file in read me mode
with ZipFile(file_name, 'r') as zip :
  print("Extracting the files...")
  zip.extractall()
  print("Done")

In [ ]:
file_name = "test1.zip"

# opening the zip file in read me mode
with ZipFile(file_name, 'r') as zip :
  print("Extracting the files...")
  zip.extractall()
  print("Done")

# Importing Libraries

In [ ]:
!pip install np_utils
!pip install sklearn

In [ ]:
import os, cv2, itertools
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.utils import shuffle

import sklearn
from sklearn.model_selection import train_test_split

# Converting image's to vectors

In [ ]:
TRAIN_DIR = './train/'
TEST_DIR = './test1/'

In [ ]:
ROWS = 64
COLS = 64
CHANNELS = 3

In [ ]:
train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]
test_images = [TEST_DIR+i for i in os.listdir(TEST_DIR)]

In [ ]:
def read_image(file_path):
  #print(file_path)
  img = cv2.imread(file_path, cv2.IMREAD_COLOR)
  #print(img)
  return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

In [ ]:
def prep_data(images):
  m = len(images)
  n_x = ROWS*COLS*CHANNELS
  
  X = np.ndarray((m,ROWS,COLS,CHANNELS), dtype=np.uint8)
  y = np.zeros((m,1))
  print("X.shape is {}".format(X.shape))
  
  for i,image_file in enumerate(images) :
    image = read_image(image_file)
    X[i,:] = np.squeeze(image.reshape((ROWS, COLS, CHANNELS)))
    if 'dog' in image_file.lower() :
      y[i,0] = 1
    elif 'cat' in image_file.lower() :
      y[i,0] = 0
    else : # for test data
      y[i,0] = image_file.split('/')[-1].split('.')[0]
      
    if i%5000 == 0 :
      print("Proceed {} of {}".format(i, m))
    
  return X,y

In [ ]:
X_train, y_train = prep_data(train_images)
X_test, y_test = prep_data(test_images)

In [ ]:
print("Train shape: {}".format(X_train.shape))
print("Test shape: {}".format(X_test.shape))

In [ ]:
X, y = shuffle(X_train, y_train)

In [ ]:
X.shape

In [ ]:
y.shape

# Splitting of dataset into train (80%), validation (20%)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

print("Train shape: {}".format(X_train.shape))
print("Train label shape: {}".format(y_train.shape))
print("Validation shape: {}".format(X_val.shape))
print("Validation label shape: {}".format(y_val.shape))

# Converting to one-hot vector representation

In [ ]:
y_train_one_hot = to_categorical(y_train)
print(y_train_one_hot.shape)

num_classes = y_train_one_hot.shape[1]
print(num_classes)

y_val_one_hot = to_categorical(y_val)
print(y_val_one_hot.shape)

# Visualizing the dataset

In [ ]:
classes = {0: 'cats',
          1: 'dogs'}

In [ ]:
def show_images(X, y, idx) :
  image = X[idx]
  #image = image.reshape((ROWS, COLS, CHANNELS))
  plt.figure(figsize=(4,2))
  plt.imshow(image)
  plt.title("This is a {}".format(classes[y[idx,0]]))
  plt.show()

In [ ]:
show_images(X_train, y_train, 0)

In [ ]:
X_train_norm = X_train / 255.
X_val_norm = X_val / 255.

In [ ]:
show_images(X_train_norm, y_train, 5)

# Final Choosen Model Architecture and Training of Model

For the details about how this architecture is choosen, scroll down to the last section on "How to choose the right model architecture iteratively?"

In [ ]:
model9 = Sequential()

model9.add(Conv2D(32, (3,3), input_shape=(ROWS, COLS, CHANNELS), activation='relu'))
model9.add(MaxPooling2D(pool_size = (2,2)))

model9.add(Conv2D(64, (3,3), activation='relu'))
model9.add(MaxPooling2D(pool_size = (2,2)))
model9.add(Dropout(0.4))

model9.add(Conv2D(128, (3,3), activation='relu'))
model9.add(MaxPooling2D(pool_size = (2,2)))
model9.add(Dropout(0.4))

model9.add(Conv2D(256, (3,3), activation='relu'))
model9.add(MaxPooling2D(pool_size = (2,2)))
model9.add(Dropout(0.4))

model9.add(Conv2D(512, (1,1), activation='relu'))
#model6.add(MaxPooling2D(pool_size = (2,2)))

model9.add(Flatten())
model9.add(Dropout(0.4))

model9.add(Dense(units=120, activation='relu'))
model9.add(Dense(units=2, activation='sigmoid'))

In [ ]:
model9.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model9.summary()

In [ ]:
model9.fit(X_train_norm, y_train_one_hot, validation_data=(X_val_norm, y_val_one_hot), epochs=50, batch_size = 64)

# Making Predictions using the model on Training and Test data

In [ ]:
image = X_train[0]
test_pred = model9.predict_classes(image.reshape(1, 64, 64, 3))

show_images(X_train, y_train,0)

print("Our Model Prediction: {}".format(test_pred))

In [ ]:
image = X_train[100]
test_pred = model9.predict_classes(image.reshape(1, 64, 64, 3))

show_images(X_train, y_train,100)

print("Our Model Prediction: {}".format(test_pred))

In [ ]:
def show_image_prediction(X, idx, model) :
  image = X[idx].reshape(1,64,64,3)
  image_class = classes[model.predict_classes(image).item()]
  image = image.reshape((ROWS, COLS, CHANNELS))
  plt.figure(figsize = (4,2))
  plt.imshow(image)
  plt.title("Test {} : I think this is {}".format(idx, image_class))
  plt.show()

In [ ]:
X_test, test_idx = prep_data(test_images)

In [ ]:
X_test_lr, test_idx = X_test, test_idx

for i in np.random.randint(0, len(X_test_lr), 10) :
  show_image_prediction(X_test_lr, i, model9)

# How to choose the right model architecture iteratively?

Choosing the right architecture and the correct set of hyperparameter is an art of Deep Learning that's very important to master for any data scientist.

Here, I give you a quick guide on how you can choose the right architecture for your particular task and the correct set of hyperparameters iteratively.

**Model 1**

1. Build a simple and rought model quickly. It will not perform good but it will give you a good insight as to where you can improve in your architecture and set of hyperparameters.

In [ ]:
model1 = Sequential()

model1.add(Conv2D(32, (3,3), input_shape=(ROWS, COLS, CHANNELS), activation='relu'))
model1.add(MaxPooling2D(pool_size = (2,2)))

model1.add(Conv2D(32, (3,3), activation='relu'))
model1.add(MaxPooling2D(pool_size = (2,2)))
model1.add(Dropout(0.6))

model1.add(Conv2D(64, (3,3), activation='relu'))
model1.add(MaxPooling2D(pool_size = (2,2)))

model1.add(Conv2D(64, (3,3), activation='relu'))
model1.add(MaxPooling2D(pool_size = (2,2)))

model1.add(Flatten())
model1.add(Dropout(0.6))

model1.add(Dense(units=120, activation='relu'))
model1.add(Dense(units=2, activation='sigmoid'))

In [ ]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model1.summary()

In [ ]:
model1.fit(X_train_norm, y_train_one_hot, validation_data=(X_val_norm, y_val_one_hot), epochs=20, batch_size = 64)

As you can see our model is showing a training accruacy of 84.84 % on training data that can be increased. However, our model right now has only 96650 trainable parameters. We can make our model more deeper to make our training accuracy even better.

**Model 2**

In [ ]:
model2 = Sequential()

model2.add(Conv2D(32, (3,3), input_shape=(ROWS, COLS, CHANNELS), activation='relu'))
model2.add(MaxPooling2D(pool_size = (2,2)))

model2.add(Conv2D(64, (3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size = (2,2)))
#model2.add(Dropout(0.6))

model2.add(Conv2D(128, (3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size = (2,2)))

model2.add(Conv2D(256, (3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size = (2,2)))

model2.add(Flatten())
#model2.add(Dropout(0.6))

model2.add(Dense(units=120, activation='relu'))
model2.add(Dense(units=2, activation='sigmoid'))

In [ ]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()

In [ ]:
model2.fit(X_train_norm, y_train_one_hot, validation_data=(X_val_norm, y_val_one_hot), epochs=20, batch_size = 64)

We can clearly see that making our model deeper do help with the training accuracy as now our Training Accuracy has increased to 98.88% but our validation accuracy is only 84.56%. This means our model is overfitting. We can add regularization techniques like dropout to improve our validation set performance and prevent overfitting of our model.

**Model 3**

In [ ]:
model3 = Sequential()

model3.add(Conv2D(32, (3,3), input_shape=(ROWS, COLS, CHANNELS), activation='relu'))
model3.add(MaxPooling2D(pool_size = (2,2)))

model3.add(Conv2D(64, (3,3), activation='relu'))
model3.add(MaxPooling2D(pool_size = (2,2)))
model3.add(Dropout(0.3))

model3.add(Conv2D(128, (3,3), activation='relu'))
model3.add(MaxPooling2D(pool_size = (2,2)))

model3.add(Conv2D(256, (3,3), activation='relu'))
model3.add(MaxPooling2D(pool_size = (2,2)))

model3.add(Flatten())
model3.add(Dropout(0.3))

model3.add(Dense(units=120, activation='relu'))
model3.add(Dense(units=2, activation='sigmoid'))

In [ ]:
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model3.summary()

In [ ]:
model3.fit(X_train_norm, y_train_one_hot, validation_data=(X_val_norm, y_val_one_hot), epochs=20, batch_size = 64)

After applying Dropout as our regularization technique, the amount of overfitting has certainly reduced but our model training accuracy has considerably reduced to 94.40%. This means that if we further increase the dropout for further reducing the overfitting, our training accuracy will decrease drastically. So, we need to increase the trainable parameters more. That means, either we can make our network more deep or remove the MaxPooling2D layer just before the flatten(). We take the 2nd approach. But both approaches can be checked. Moreover, we add the dropout as well so that our model do not overfit.

Model 4

In [ ]:
model4 = Sequential()

model4.add(Conv2D(32, (3,3), input_shape=(ROWS, COLS, CHANNELS), activation='relu'))
model4.add(MaxPooling2D(pool_size = (2,2)))

model4.add(Conv2D(64, (3,3), activation='relu'))
model4.add(MaxPooling2D(pool_size = (2,2)))
model4.add(Dropout(0.4))

model4.add(Conv2D(128, (3,3), activation='relu'))
model4.add(MaxPooling2D(pool_size = (2,2)))
model4.add(Dropout(0.4))

model4.add(Conv2D(256, (3,3), activation='relu'))
model4.add(MaxPooling2D(pool_size = (2,2)))
model4.add(Dropout(0.4))

model4.add(Conv2D(512, (1,1), activation='relu'))
#model6.add(MaxPooling2D(pool_size = (2,2)))

model4.add(Flatten())
model4.add(Dropout(0.4))

model4.add(Dense(units=120, activation='relu'))
model4.add(Dense(units=2, activation='sigmoid'))

This model gives the best accuracy. So we trained it for 50 epochs and the result for which is shown in the "Final Choosen Model Architecture and Training of Model" section.

You can further twerk the hyperparameter and the model architecture to get even better accuracy. However, I will leave you here.

Wish You A Happy Exploring!!